In [10]:
import pandas as pd

# Load the data from the "Punti" sheet
data = pd.read_excel("FantaSPL_V2.2.xlsx", sheet_name="Punti")

# Display the first few rows to understand the structure
data.head()

,Data,Player,Ruolo,Colore Squadra,Vincitori,Vinta,#Goal,Goal Subiti,Goal Fatti,AutoGoal,...,Penalità.1,Punteggio Difesa,Punteggio Centrocampisti,Squadra Vincente,#Goal.1,AutoGoal.1,SPL Bonus.1,MVP.1,Porta Amico,TOTAL
0,2023-01-12,Andrea Limonta,Difensore,Scuri,Bianchi,No,0,18,13,0,...,0,0,0,0,0,0,3,0,0,7
1,2023-01-12,Luca Stoppi,Difensore,Scuri,Bianchi,No,2,18,13,1,...,0,0,0,0,2,-2,0,0,0,4
2,2023-01-12,Riccardo Ricci,Centrocampista,Scuri,Bianchi,No,0,18,13,0,...,0,0,0,0,0,0,0,0,0,4
3,2023-01-12,Marco Taglio,Centrocampista,Scuri,Bianchi,No,3,18,13,0,...,0,0,0,0,3,0,0,0,0,7
4,2023-01-12,Mazzu,Centrocampista,Scuri,Bianchi,No,7,18,13,0,...,0,0,0,0,7,0,0,0,0,11


In [11]:
# Calculate the average score for each player
avg_scores = data.groupby('Player')['TOTAL'].mean().sort_values(ascending=False)

# Display the average scores
avg_scores.head()

Player
Giacomo (Stoppi)    17.000000
Matteo (Mazzu)      17.000000
Amico Gio           16.000000
Cerro               15.750000
Matteo Virg         15.666667
Name: TOTAL, dtype: float64

In [12]:
# Define the quantile values
quantiles = [0, 0.2, 0.4, 0.6, 0.8, 1]

# Calculate quantile thresholds for categorization
quantile_thresholds = avg_scores.quantile(quantiles).to_dict()

# Create a function to categorize players
def categorize_player(score):
    if score >= quantile_thresholds[0.8]:
        return "Top Performers"
    elif score >= quantile_thresholds[0.6]:
        return "Above Average"
    elif score >= quantile_thresholds[0.4]:
        return "Average Players"
    elif score >= quantile_thresholds[0.2]:
        return "Below Average"
    else:
        return "Lowest Performers"

# Apply the categorization function to each player's average score
categories = avg_scores.apply(categorize_player)

# Check the number of players in each category to ensure a minimum of 5 players
category_counts = categories.value_counts()

category_counts

TOTAL
Below Average        14
Top Performers       13
Above Average        13
Average Players      12
Lowest Performers    12
Name: count, dtype: int64

In [14]:
# Define the price bands for each category based on the $100 budget for 7 players
price_bands = {
    "Top Performers": 18,   # Highest price
    "Above Average": 15,    # Slightly lower than the top performers
    "Average Players": 12,  # Mid-range price
    "Below Average": 9,     # Slightly lower than average players
    "Lowest Performers": 6  # Lowest price
}

# Assign the price to each player based on their category
player_prices = categories.map(price_bands)

# Create a dataframe with player names, their average scores, categories, and prices
final_df = pd.DataFrame({
    'Average Score': avg_scores,
    'Category': categories,
    'Price ($)': player_prices
}).reset_index()

final_df

,Player,Average Score,Category,Price ($)
0,Giacomo (Stoppi),17.000000,Top Performers,18
1,Matteo (Mazzu),17.000000,Top Performers,18
2,Amico Gio,16.000000,Top Performers,18
3,Cerro,15.750000,Top Performers,18
4,Matteo Virg,15.666667,Top Performers,18
...,...,...,...,...
59,Amico Matteo,5.000000,Lowest Performers,6
60,Puddi,5.000000,Lowest Performers,6
61,Ale (Fabri),4.000000,Lowest Performers,6
62,Braz,4.000000,Lowest Performers,6


In [15]:
# Load the data from the "Summary" sheet
summary_data = pd.read_excel("FantaSPL_V2.2.xlsx", sheet_name="Summary")

# Display the first few rows to understand the structure
summary_data.head()

,Unnamed: 0,Player,Penalità,Amici Portati,Ruolo,Vinte,Partite,Autogoal,MVP,SPL Bonus,...,Goal,Punti/Partita,Ultima Partita,TOTAL,Prev. Game,Prev. Rank,Pos. New,Unnamed: 21,Last Game,2023-07-06 00:00:00
0,1,Daniele (Francesco),0,1,Centrocampista,12,17,0,0,0,...,61,14.941176,23,254,231,1,0,NaN,NaN,NaN
1,2,Luca Stoppi,1,4,Difensore,8,23,2,1,1,...,18,10.173913,9,234,225,3,1,NaN,NaN,NaN
2,3,Cameron McAinsh,0,1,Centrocampista,9,22,0,0,0,...,40,10.409091,0,229,229,2,-1,NaN,NaN,NaN
3,4,Andrea Limonta,0,2,Difensore,11,22,1,0,1,...,3,10.363636,4,228,224,4,0,NaN,NaN,NaN
4,5,Giovanni Aiello,0,0,Difensore,11,21,0,0,2,...,12,10.761905,15,226,211,6,1,NaN,NaN,NaN


In [16]:
# Aggregating the necessary columns to recreate the summary table
summary_agg = data.groupby('Player').agg({
    'Penalità': 'sum',
    'Porta Amico': 'sum',
    'Ruolo': 'first',  # Taking the first value as role should be consistent for a player
    'Vinta': lambda x: (x == 'Si').sum(),
    'Data': 'count',
    'AutoGoal': 'sum',
    'MVP': 'sum',
    'SPL Bonus': 'sum',
    '#Goal': 'sum',
    'TOTAL': ['sum', 'mean', 'last']
}).reset_index()

# Rename columns for clarity
summary_agg.columns = ['Player', 'Penalità', 'Amici Portati', 'Ruolo', 'Vinte', 'Partite', 
                       'Autogoal', 'MVP', 'SPL Bonus', 'Goal', 'TOTAL', 'Punti/Partita', 'Ultima Partita']

# Sort by total points in descending order
summary_agg = summary_agg.sort_values(by='TOTAL', ascending=False)

# Reset index
summary_agg.reset_index(drop=True, inplace=True)

# Display the recreated summary table
summary_agg.head()


,Player,Penalità,Amici Portati,Ruolo,Vinte,Partite,Autogoal,MVP,SPL Bonus,Goal,TOTAL,Punti/Partita,Ultima Partita
0,Daniele (Francesco),NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,3,Centrocampista,0,17,0,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,61,254,14.941176,14
1,Luca Stoppi,NoNoNoSìNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,12,Difensore,0,23,2,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoSìNoNoNoNoNoNoNo,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoSìNoNoNoNoNoNo,18,234,10.173913,11
2,Cameron McAinsh,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,3,Centrocampista,0,22,0,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,40,229,10.409091,10
3,Andrea Limonta,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,6,Difensore,0,22,1,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,SìNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,3,228,10.363636,15
4,Giovanni Aiello,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,0,Difensore,0,21,0,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo,NoNoNoNoSìNoNoSìNoNoNoNoNoNoNoNoNoNoNoNoNo,12,226,10.761905,15
